In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pickle
import numpy as np
import math
from mosestokenizer import *
from collections import Counter

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [4]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [5]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


In [6]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("forme : ",pairs[0])
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
forme :  ['va !', 'go .']
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['j ai sommeil alors je pars maintenant .', 'i m sleepy so i am leaving now .']


In [7]:
from torchvision import models
from torchsummary import summary


/Users/tarekbenaissa/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/tarekbenaissa/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <681C00E9-5647-3069-B4FB-5DBB8737D1E1> /Users/tarekbenaissa/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <479B05C1-363C-3799-AAF4-CF79A0358897> /Users/tarekbenaissa/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/torch/lib/libc10.dylib
  warn(f"Failed to load image Python extension: {e}")


In [9]:
"""
device = 'cpu'
source_vocab_size = 5000 ; # nombre de mots dans la phrase!! #dans le dataset
embedding_dim = 128
hidden_dim = 1000
test_encoder = EncoderRNN( embedding_dim, hidden_dim)

a = torch.zeros((32, 5))
"""

"\ndevice = 'cpu'\nsource_vocab_size = 5000 ; # nombre de mots dans la phrase!! #dans le dataset\nembedding_dim = 128\nhidden_dim = 1000\ntest_encoder = EncoderRNN( embedding_dim, hidden_dim)\n\na = torch.zeros((32, 5))\n"

In [11]:
#output_seq, hid = test_encoder(a)

In [12]:
"""
class EncoderRNN(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super(EncoderRNN, self).__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.hidden_size = enc_hid_dim
    def forward(self, src):

        # src= [src len, batch size]

        embedded = self.dropout(self.embedding(src).view(1, 1, -1))

        # embedded = [src len, batch size, emb dim]

        output, hidden = self.rnn(embedded)
        output = output.reshape((2,1,self.hidden_size))
        
        print("output.size = ", output.size())

        # outputs = [src len, batch size, hid dim * num directions]
        # hidden = [n layers * num directions, batch size, hid dim]

        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer

        # hidden [-2, :, : ] is the last of the forwards RNN
        # hidden [-1, :, : ] is the last of the backwards RNN

        # 初始译码器隐藏是向前和向后的最终隐藏状态
        # 编码器RNNs通过一个线性层
       
        hidden = torch.tanh((torch.cat((hidden[0,:, :], hidden[1,:, :]), dim =1)))
        print("hidden size : ",hidden.size())
        #hidden = torch.tanh(hidden)

        # outputs = [src len, batch size, enc hid dim * 2]
        # hidden = [batch size, dec hid dim]
        print(output.size())
        return output, hidden
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size*2, dtype=torch.int32, device=device)
        
"""

'\nclass EncoderRNN(nn.Module):\n    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):\n        super(EncoderRNN, self).__init__()\n\n        self.embedding = nn.Embedding(input_dim, emb_dim)\n\n        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)\n\n        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)\n        \n        self.dropout = nn.Dropout(dropout)\n        \n        self.hidden_size = enc_hid_dim\n    def forward(self, src):\n\n        # src= [src len, batch size]\n\n        embedded = self.dropout(self.embedding(src).view(1, 1, -1))\n\n        # embedded = [src len, batch size, emb dim]\n\n        output, hidden = self.rnn(embedded)\n        output = output.reshape((2,1,self.hidden_size))\n        \n        print("output.size = ", output.size())\n\n        # outputs = [src len, batch size, hid dim * num directions]\n        # hidden = [n layers * num directions, batch size, hid dim]\n\n        # hidden is stacked [forward_1, backwar

In [446]:

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        # input_size : taille du vocabulaire français
        # hidden
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size,bidirectional=True)
        
        #self.fc_hidden = nn.Linear(hidden_size*2, hidden_size)
        #self.fc_output = nn.Linear(hidden_size*2, hidden_size)

    def forward(self, input):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output)
        #print("hidden avant reshape : ",hidden.size())
        #print("output avant reshape : ",output.size())
        #print("output 512 : ", output)
        
        # output : nous donne la sortie des cellule de notre réseau RNN donc les yi --> c des code = token
        # dimension : [lenth of sequence, batch size, 2*hidden_size] : quand c bidirectionnel
        
        # hidden : contient le dernier état de la mémoire de chaque cellule du RNN
        # dimension : [2*number of layers, hidden_size]
        
        hidden = hidden.reshape((1,1,2*self.hidden_size))
        #hidden = self.fc_hidden(hidden)
        #output = self.fc_output(output)
        #print("output 256 : ", output)
        #print("hidden apres reshape : ",hidden.size())
        #print("output apres reshape : ",encoder_states.size())
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, dtype=torch.int32, device=device)


In [447]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size*3 , self.max_length)  #max_length) 
        self.attn_combine = nn.Linear(self.hidden_size *3, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size*2, self.hidden_size*2)
        self.out = nn.Linear(self.hidden_size*2, self.output_size)
        

    def forward(self, input, hidden, encoder_outputs):
        #print("-----------------------------")
        #print("hidden donné au decodeur size : ",input.size())
        #print("logique psq on passe mot par mot")
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        
        #print("size embedded donner par l encodeur : ", embedded[0].size())
        #print("size hidden donner par l encodeur : ", hidden[0].size())
        #print("size en entree de l attention : ", (torch.cat((embedded[0], hidden[0]), 1)).size())
        #print("self.attn(torch.cat((embedded[0], hidden[0]), 1)) : ", self.attn(torch.cat((embedded[0], hidden[0]), 1)).size())
        #print("attn_weights size : ", attn_weights.size())
        #print("attn_weights.unsqueeze(0) size : ", attn_weights.unsqueeze(0).size())
        #print("encoder_outputs.unsqueeze(0) size : ", encoder_outputs.unsqueeze(0).size())
        
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
        
        #print(" context vector attn_applied sortie du bmm : ", attn_applied.size())
        
        #output = torch.cat((embedded[0], attn_applied[0]), 1)
        #print("context vector : ", output.size())
        #output = self.attn_combine(output).unsqueeze(0)
        #print("self.attn_combine size: ", output.size())
        output = attn_applied 

        output = F.relu(output)
        #print("decoder : output de l'attention size -- a l entre du gru : ",output.size())
        #print("decoder : hidden size -- a l entre du gru : ",hidden.size())
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size*2, device=device)

In [448]:
"""
# sans attention
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        # output_size : taille du vocabulaire anglais
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        # à l entrée du décodeur on a 2*¨hidden_size pour les memoire forward et backward + embidding_size = hidden_size : taille embedding du dernier output predit
        self.gru = nn.GRU(3*hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
"""

'\n# sans attention\nclass DecoderRNN(nn.Module):\n    def __init__(self, hidden_size, output_size):\n        # output_size : taille du vocabulaire anglais\n        super(DecoderRNN, self).__init__()\n        self.hidden_size = hidden_size\n\n        self.embedding = nn.Embedding(output_size, hidden_size)\n        # à l entrée du décodeur on a 2*¨hidden_size pour les memoire forward et backward + embidding_size = hidden_size : taille embedding du dernier output predit\n        self.gru = nn.GRU(3*hidden_size, hidden_size)\n        self.out = nn.Linear(hidden_size, output_size)\n        self.softmax = nn.LogSoftmax(dim=1)\n\n    def forward(self, input, hidden):\n        \n        output = self.embedding(input).view(1, 1, -1)\n        output = F.relu(output)\n        output, hidden = self.gru(output, hidden)\n        output = self.softmax(self.out(output[0]))\n        return output, hidden\n\n    def initHidden(self):\n        return torch.zeros(1, 1, self.hidden_size, device=device)\n'

In [449]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [450]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size*2, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei])
        #print("input_tensor[ei] size : ",input_tensor[ei].size())
        #print("encoder_outputs[ei] size : ",encoder_outputs[ei].size())
        #print(" encoder_output[0, 0] size : ", encoder_output[0, 0].size())
        encoder_outputs[ei] = encoder_output[0, 0]
        
    
    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [451]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [452]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [453]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [481]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size*2, device=device)

        for ei in range(input_length):
            
            #encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden, encoder.hidden_size)
            #encoder_outputs[ei] += encoder_output[0, 0]
            
            encoder_output, encoder_hidden = encoder(input_tensor[ei])
            #print("input_tensor[ei] size : ",input_tensor[ei].size())
            #print("encoder_outputs[ei] size : ",encoder_outputs[ei].size())
            #print(" encoder_output[0, 0] size : ", encoder_output[0, 0].size())
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [482]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [483]:
hidden_size = 256
"""
if torch.cuda.is_available():
    torch.device('cuda')
else :   
    torch_device('cpu')
"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

#encoder1 = EncoderRNN(input_dim = input_lang.n_words,emb_dim =256,enc_hid_dim = 1000, dec_hid_dim = 1000, dropout = 0.6) 
encoder1 = EncoderRNN(input_size = input_lang.n_words, hidden_size = hidden_size)
#trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

In [484]:
# sauvegarde du modèle
#torch.save(encoder1,"encoder1_bi.pt")
#torch.save(attn_decoder1,"attn_decoder1_bi.pt")


In [485]:
encoder1_loaded = torch.load("encoder1_bi.pt")
attn_decoder1_loaded = torch.load("attn_decoder1_bi.pt")


In [486]:
evaluateRandomly(encoder1_loaded, attn_decoder1_loaded)

> il est trop stupide pour craindre le danger .
= he is too dumb to fear danger .
< he s too stupid to fear danger . <EOS>

> vous etes garee en double file .
= you re double parked .
< you re very parked . <EOS>

> je suis le chef de cette equipe .
= i am the leader of this team .
< i m your of of this team . <EOS>

> je manque d argent .
= i am short of money .
< i m out of money . <EOS>

> elle est hors de danger .
= she is out of danger .
< she is ashamed of danger . <EOS>

> je ne suis pas patiente .
= i m not patient .
< i m not patient . <EOS>

> je rencontre quelqu un pour diner .
= i m meeting someone for dinner .
< i m going someone for dinner . <EOS>

> vous etes un petit menteur .
= you re a little liar .
< you re a little liar . <EOS>

> il te regarde .
= he s looking at you .
< he s a . <EOS>

> je ne vais pas livrer de noms .
= i m not going to name names .
< i m not of of name . <EOS>



In [487]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "je suis trop froid .")
plt.matshow(attentions.numpy())

In [31]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("elle a cinq ans de moins que moi .")

evaluateAndShowAttention("elle est trop petit .")

evaluateAndShowAttention("je ne crains pas de mourir .")

evaluateAndShowAttention("c est un jeune directeur plein de talent .")

input = elle a cinq ans de moins que moi .
output = she s five years younger than me . <EOS>
input = elle est trop petit .
output = she s too short . <EOS>
input = je ne crains pas de mourir .
output = i m not afraid to die . <EOS>
input = c est un jeune directeur plein de talent .
output = he s a talented young man . <EOS>


/var/folders/fp/bdcgs2110sxb3y211lr1nqgr0000gn/T/ipykernel_90679/744067545.py:9: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
/var/folders/fp/bdcgs2110sxb3y211lr1nqgr0000gn/T/ipykernel_90679/744067545.py:11: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)
/var/folders/fp/bdcgs2110sxb3y211lr1nqgr0000gn/T/ipykernel_90679/744067545.py:17: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
